In [ ]:
# Celda 1: Configuración de MLflow para Codespaces
import os
import socket
from contextlib import closing
import mlflow

def find_free_port():
    """Encuentra un puerto disponible automáticamente"""
    with closing(socket.socket(socket.AF_INET, socket.SOCK_STREAM)) as s:
        s.bind(('', 0))
        s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        return s.getsockname()[1]

# Configurar MLflow
mlflow_port = find_free_port()
mlflow_dir = os.path.join(os.getcwd(), "mlruns")
os.makedirs(mlflow_dir, exist_ok=True)

# Iniciar MLflow UI en segundo plano
mlflow.set_tracking_uri(f"file:{mlflow_dir}")
get_ipython().system_raw(f"mlflow ui --port {mlflow_port} --host 0.0.0.0 --serve-artifacts &")

# Mostrar URL de acceso
print(f"\n MLflow UI disponible en: https://{os.getenv('CODESPACE_NAME')}-{mlflow_port}.{os.getenv('GITHUB_CODESPACES_PORT_FORWARDING_DOMAIN')}\n")



In [ ]:
import pandas as pd
import numpy as np

#leer el data
import os

try:
    diabetes_cleaned = pd.read_csv('data/diabetes_cleaned.csv')
    print("Datos cargados correctamente")
except FileNotFoundError:
    raise Exception("Error: Ejecuta primero el notebook de Análisis Exploratorio")

In [ ]:

# 5. Dividir los datos
X = diabetes_cleaned[['hbA1c_level', 'blood_glucose_level', 'age', 'bmi']]
y = diabetes_cleaned['diabetes']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Iniciar un run y registrar todo
with mlflow.start_run():
    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

#metricas
    acc = accuracy_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("mae", mae)
    mlflow.log_param("random_state", 42)

    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("mse", mse)


    # Guardar modelo
    model_path = "modelo_rf.pkl"
    joblib.dump(model, model_path)
    mlflow.log_artifact(model_path)

    # Guardar dataset
    diabetes_cleaned.to_csv("diabetes_logged.csv", index=False)
    mlflow.log_artifact("diabetes_logged.csv")

    # Curva ROC
    y_prob = model.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_prob)
    plt.figure()
    plt.plot(fpr, tpr, label="ROC Curve")
    plt.xlabel("FPR")
    plt.ylabel("TPR")
    plt.title("Curva ROC")
    plt.legend()
    plt.savefig("roc_curve.png")
    mlflow.log_artifact("roc_curve.png")

    # Matriz de confusión
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(cm)
    disp.plot()
    plt.title("Matriz de Confusión")
    plt.savefig("confusion_matrix.png")
    mlflow.log_artifact("confusion_matrix.png")

    # Guardar predicciones
    pred_df = pd.DataFrame({"y_test": y_test, "y_pred": y_pred})
    pred_df.to_csv("predicciones.csv", index=False)
    mlflow.log_artifact("predicciones.csv")


    mlflow.sklearn.log_model(model, "modelo_random_forest", input_example=X.iloc[:2])





